# Analysis of IMR90 Datasets with Various Metrics and Norms

In [ ]:
import sys
sys.path.append('../')

import numpy as np
from AnalysisTools import Ana, PlotHelper


# Initialize the analysis object
analysis = Ana(showPlots=True, execution_mode='cpu')

# Add datasets
analysis.add_dataset(label="IMR90SIM", folder="data/IMR90SIM")
analysis.add_dataset(label='IMR90OPT', folder='data/IMR90OPT')
analysis.add_dataset(label="IMR90OPT57", folder='data/IMR90OPT56')

# Process trajectories for each dataset
analysis.process_trajectories(label="IMR90SIM", filename="traj_chr_IMR90OPT_0.cndb", folder_pattern=['iteration_', [1, 20]])
analysis.process_trajectories(label="IMR90OPT", filename="traj_0.cndb", folder_pattern=['iteration_', [1, 20]])
analysis.process_trajectories(label="IMR90OPT57", filename="traj_0.cndb", folder_pattern=['iteration_', [1, 20]])

# Initialize PlotHelper
pl = PlotHelper()

## Spectral Clustering Analysis with Default Parameters

In [ ]:
cluster_results_default = analysis.spectral_clustering("IMR90OPT", "IMR90SIM", "IMR90OPT57")
# pl.plot(plot_type="spectralclusteringplot", data=[X_default, cluster_results_default], plot_params={"title": "Default Spectral Clustering"})

## Spectral Clustering Analysis with Different Norms

In [ ]:
norms = ["ice", "log_transform", "vc", "kr"]

for norm in norms:
    cluster_results_norm = analysis.spectral_clustering("IMR90OPT", "IMR90SIM", "IMR90OPT57", norm=norm)


## Spectral Clustering Analysis with Different Metrics

In [ ]:
metrics = ["euclidean", "pearsons", "spearman", "contact", "log2_contact"]

for metric in metrics:
    cluster_results_metric = analysis.spectral_clustering("IMR90OPT", "IMR90SIM", "IMR90OPT57", metric=metric)


## PCA Analysis

In [ ]:
prinComp_pca, expVarRatio_pca, fclust_pca = analysis.pca("IMR90OPT", "IMR90SIM", "IMR90OPT57")
pl.plot(plot_type="pcaplot", data=[prinComp_pca, expVarRatio_pca, fclust_pca], plot_params={"title": "PCA Plot"})

## t-SNE Analysis

In [ ]:
tsne_res, fclust_tsne = analysis.tsne("IMR90OPT", "IMR90SIM", "IMR90OPT57")
pl.plot(plot_type="tsneplot", data=[tsne_res, fclust_tsne], plot_params={"title": "t-SNE Plot"})

## UMAP Analysis

In [ ]:
umap_res, fclust_umap = analysis.umap("IMR90OPT", "IMR90SIM", "IMR90OPT57")
pl.plot(plot_type="umapplot", data=[umap_res, fclust_umap], plot_params={"title": "UMAP Plot"})

## IVIS Analysis

In [ ]:
ivis_res, fclust_ivis = analysis.ivis_clustering("IMR90OPT", "IMR90SIM", "IMR90OPT57")
pl.plot(plot_type="ivisplot", data=[ivis_res, fclust_ivis], plot_params={"title": "IVIS Plot"})

## Dendrogram

In [ ]:
Z_dendro = analysis.dendogram_Z("IMR90OPT", "IMR90SIM", "IMR90OPT57")
pl.plot(plot_type="dendrogram", data=Z_dendro, plot_params={"title": "Dendrogram"})

## Distance Map

In [ ]:
for dataset in ["IMR90OPT", "IMR90SIM", "IMR90OPT57"]:
    dist_map = analysis.dist_map(dataset)
    pl.plot(plot_type="euclidiandistmap", data=[dist_map], plot_params={"title": f"{dataset} Distance Map"})